## Script to move raw MACS files based on extracted footprints files 
**author:** Ingmar Nitze, Tabea Rettelbach

**contact:** ingmar.nitze@awi.de

**version date:** 2021-11-29


### Imports 

In [1]:
import geopandas as gpd
import shutil
import os
import glob
import pandas as pd
from IPython.display import clear_output
import sys
import numpy as np
import tqdm
import zipfile
from pathlib import Path
from joblib import delayed, Parallel

In [2]:
pwd = Path(os.getcwd())

### Functions 

In [3]:
def prepare_df_for_mipps2(path_footprints, path_infiles):
    # Load filtered footprints files
    df = gpd.read_file(path_footprints)
    
    flist = list(Path(path_infiles).glob('*/*.macs'))
    flist = [f'"{str(f)}"' for f in flist]
    
    df_full = pd.DataFrame()
    df_full['full_path'] = flist
    df_full['basename'] = pd.DataFrame(df_full['full_path'].apply(lambda x: os.path.basename(x)))
    # return Inner join of lists - create filtered list of filepaths 
    return df.set_index('Basename').join(df_full.set_index('basename'))

In [4]:
def prepare_df_for_mipps(path_footprints, path_infiles):
    # Load filtered footprints files
    df = gpd.read_file(path_footprints)
    
    #flist = glob.glob(path_infiles + '/*/*.macs')
    flist = list(Path(path_infiles).glob('*/*.macs'))
    flist = [str(f) for f in flist]
    
    df_full = pd.DataFrame()
    df_full['full_path'] = flist
    df_full['basename'] = pd.DataFrame(df_full['full_path'].apply(lambda x: os.path.basename(x)))
    # return Inner join of lists - create filtered list of filepaths 
    return df.set_index('Basename').join(df_full.set_index('basename'))

def write_exif(outdir, tag, exifpath):
    s = f'{exifpath} -overwrite_original -Model="{tag}" {outdir}'
    print(s)
    os.system(s)

### Settings 
* prefer full/absolute paths
* Create processing template automatically

#### Variable Settings 

In [5]:
# Set project directory here, where you want to process your data
PROJECT_DIR = r'G:\99_tabea\05_anaktuvuk_07d' # SET Project output

# Set raw data dir here for the speicific image acquisition project
path_infiles = r'N:\response\Restricted_Airborne\MACs\Alaska\ThawTrend-Air_2019\raw_data\20190723-011256_3S_Anaktuvuk_Fire_Scar_flight_plan_1650ft'

# determine which sensors to include in processing (possible options: 'left', 'right', 'nir')
sensors = ['left', 'right']

#### Fixed Settings 

In [6]:
#CODE_DIR = r'D:\pix4d_Processing\code\MACS_tools'
CODE_DIR = pwd
MIPPS_DIR = r'C:\Program Files\DLR MACS-Box\bin'
MIPPS_BIN = r'..\tools\MACS\mipps.exe'
EXIF_PATH = Path(CODE_DIR / Path(r'exiftool\exiftool.exe'))

In [7]:
#mipps_script_nir = r'mipps_scripts\33552_all_taps_2018-09-26_12-58-15_modelbased.mipps'
#mipps_script_right = r'mipps_scripts\33576_all_taps_2018-09-26_13-13-43_modelbased.mipps'
#mipps_script_left = r'mipps_scripts\33577_all_taps_2018-09-26_13-21-24_modelbased.mipps'

mipps_script_nir = r'mipps_scripts\33552_all_taps_2018-09-26_12-58-15_modelbased_x4.mipps'
mipps_script_right = r'mipps_scripts\33576_all_taps_2018-09-26_13-13-43_modelbased_x4.mipps'
mipps_script_left = r'mipps_scripts\33577_all_taps_2018-09-26_13-21-24_modelbased_x4.mipps'


mipps_script_nir = pwd / mipps_script_nir
mipps_script_right = pwd / mipps_script_right
mipps_script_left = pwd / mipps_script_left


In [8]:
DATA_DIR = Path(PROJECT_DIR / Path(r'01_rawdata\tif'))
OUTDIR = {'right': DATA_DIR / Path('33576_Right'),
          'left':DATA_DIR / Path('33577_Left'),
          'nir':DATA_DIR / Path('33552_NIR')}
tag = {'right':'MACS_RGB_Right_33576',
       'left':'MACS_RGB_Left_33577',
       'nir':'MACS_NIR_33552'}

In [9]:
# Path of filtered footprints file (.shp file)
path_footprints = os.path.join(PROJECT_DIR, '02_studysites','footprints.shp')
outdir = os.path.join(PROJECT_DIR, '01_rawdata','tif')

#### Prepare processing dir 
* check if exists

In [10]:
zippath = os.path.join(CODE_DIR, 'processing_folder_structure_template.zip')
nav_script_path = os.path.join(CODE_DIR, 'pix4dnav.py')

In [11]:
with zipfile.ZipFile(zippath, 'r') as zip_ref:
    zip_ref.extractall(PROJECT_DIR)
shutil.copy(nav_script_path, outdir)

'G:\\99_tabea\\05_anaktuvuk_07d\\01_rawdata\\tif\\pix4dnav.py'

### Manual Selection of footprints 

Now select footprints and export selection as ***footprints.shp*** to ***02_footprints*** in your working directory

#### Load filtered footprints file 

In [12]:
#df_final = prepare_df_for_mipps(path_footprints, path_infiles)
df_final = prepare_df_for_mipps(path_footprints, path_infiles)

#### Workaround to deal with spaces in path" 

In [13]:
df_final['full_path'] = df_final.apply(lambda x: f'"{x.full_path}"', axis=1)

In [14]:
print("Total number of images:", len(df_final))
print("NIR images:", (df_final['Looking'] == 'center').sum())
print("RGB right images:", (df_final['Looking'] == 'right').sum())
print("RGB left images:", (df_final['Looking'] == 'left').sum())

Total number of images: 4949
NIR images: 1745
RGB right images: 1691
RGB left images: 1513


#### Run Process 

In [15]:
os.chdir(MIPPS_DIR)

In [16]:
max_roll = 3 # Select maximum roll angle to avoid image issues
chunksize = 20 # this is a mipps-script thing

In [17]:
# this is relevant for NIR only
if 'nir' in sensors:
    looking = 'center'
    q = (np.abs(df_final['Roll[deg]']) < max_roll) & (df_final['Looking'] == looking)
    df_nir = df_final[q]
    print(len(df_nir))
    for df in tqdm.tqdm_notebook(np.array_split(df_nir, len(df_nir) // chunksize)):
        outlist = ' '.join(df['full_path'].values[:])
        s = f'{MIPPS_BIN} -c={mipps_script_nir} -o={outdir} -j=4 {outlist}'
        os.system(s)
        #print(s)

In [18]:
# this is RGB
if 'right' in sensors:
    looking = 'right'
    q = (np.abs(df_final['Roll[deg]']) < max_roll) & (df_final['Looking'] == looking)
    df_right = df_final[q]
    for df in tqdm.tqdm_notebook(np.array_split(df_right, len(df_right) // chunksize)):
        outlist = ' '.join(df['full_path'].values[:])
        s = f'{MIPPS_BIN} -c={mipps_script_right} -o={outdir} -j=4 {outlist}'
        os.system(s)

C:\Users\trettel\.conda\envs\macs_tools\lib\site-packages\ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/83 [00:00<?, ?it/s]

In [19]:
if 'left' in sensors:
    looking = 'left'
    q = (np.abs(df_final['Roll[deg]']) < max_roll) & (df_final['Looking'] == looking)
    df_left = df_final[q]
    for df in tqdm.tqdm_notebook(np.array_split(df_left, len(df_left) // chunksize)):
        outlist = ' '.join(df['full_path'].values[:])
        s = f'{MIPPS_BIN} -c={mipps_script_left} -o={outdir} -j=4 {outlist}'
        os.system(s)

C:\Users\trettel\.conda\envs\macs_tools\lib\site-packages\ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/74 [00:00<?, ?it/s]

#### Write exif information into all images 

In [20]:
for sensor in tqdm.tqdm_notebook(sensors):
    print(sensor)
    %time write_exif(OUTDIR[sensor], tag[sensor], EXIF_PATH)

C:\Users\trettel\.conda\envs\macs_tools\lib\site-packages\ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


  0%|          | 0/2 [00:00<?, ?it/s]

left
D:\pix4d\MACS_tools\exiftool\exiftool.exe -overwrite_original -Model="MACS_RGB_Left_33577" G:\99_tabea\05_anaktuvuk_07d\01_rawdata\tif\33577_Left
Wall time: 11min 6s
right
D:\pix4d\MACS_tools\exiftool\exiftool.exe -overwrite_original -Model="MACS_RGB_Right_33576" G:\99_tabea\05_anaktuvuk_07d\01_rawdata\tif\33576_Right
Wall time: 13min 22s


#### Nav

In [21]:
navfile = list(Path(path_infiles).glob('*nav.txt'))[0]

In [22]:
shutil.copy(navfile, OUTDIR['nir'].parent / 'nav.txt')

WindowsPath('G:/99_tabea/05_anaktuvuk_07d/01_rawdata/tif/nav.txt')

In [23]:
os.chdir(OUTDIR['nir'].parent)
os.system('python pix4dnav.py')

0